In [23]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt


In [24]:
raw_df = pd.read_csv("anes_timeseries_cdf_csv_20211118.csv", low_memory=False)

In [45]:
running = {}
years_dict = {}

In [47]:
#Congressional Dataframe's (3)

#House
df = pd.read_csv('/Users/anujchaturvedi/research/dw_nominate/vvpd.csv')
dfHouse = df.loc[df['chamber'] == 'House']

dfHouse1980 = dfHouse.loc[dfHouse['year'] > 1979]

years = dfHouse1980['year']

dfHouseBlack1980 = []
for y in range(len(dfHouse1980)):
    dfHouseBlack1980.append(dfHouse1980['rep.mean.d1'][y + 51] - dfHouse1980['dem.mean.d1'][y + 51])

#Senate
dfSenate = df.loc[df['chamber'] == 'Senate']
dfSenate1980 = dfSenate.loc[dfSenate['year'] > 1979]

dfSenateBlack1980 = []
for y in range(len(dfSenate1980)):
    dfSenateBlack1980.append(dfSenate1980['rep.mean.d1'][y + 123] - dfSenate1980['dem.mean.d1'][y + 123])

#Polarization
dfChamberDiff = []
for x in range(len(dfHouseBlack1980)):
    dfChamberDiff.append(dfHouseBlack1980[x] + dfSenateBlack1980[x])
running['Chamber Polarization DWNominate'] = dfChamberDiff
years_dict['Chamber Polarization DWNominate'] = years
print(dfChamberDiff)

[1.2164802094887905, 1.2358667730976083, 1.2628485988305922, 1.2528986883821704, 1.268948369076917, 1.277750320309245, 1.3380349364791286, 1.410717439058117, 1.4612308026355196, 1.4354614746449712, 1.4498226053451386, 1.442830233491998, 1.4958840472643846, 1.4948165735754302, 1.5109576809672487, 1.604944251058914, 1.664411992417104, 1.6892642034025867, 1.7072411969696968, 1.7140947928847634, 1.7587899116235641]


In [48]:
# Subset

vars = {
    "VCF0004": "year",
    "VCF0301": "party",
    "VCF0803": "ideo",
    "VCF0009z": "weightz",
    "VCF0211": "thermo_liberals",
    "VCF0212": "thermo_conservatives",
    "VCF0218": "thermo_democratparty",
    "VCF0224": "thermo_republicanparty",
    "VCF0830": "abram_civil_aid_to_blacks",
    # "VCF0803": "abram_ideo", # Note: cannot use VCF0803 twice (for ideo and abram_ideo) here, so added in pre-processing block
    "VCF0843": "abram_foreign_defense_spending",
    "VCF0809": "abram_econ_guarantee_jobs",
    "VCF0806": "abram_econ_health_insurance",
    "VCF0839": "abram_econ_gov_services",
    "VCF0838": "abram_moral_abortion_allowed_law",
}
df = raw_df.loc[:, vars.keys()]
df.rename(columns=vars, inplace=True)

In [49]:
# Preprocessing

orig_ideo = df["ideo"]

df["party"] = df["party"].replace([" ", "0"], np.nan).transform(lambda x: int(x) - 4 if not pd.isnull(x) else np.nan)
df["ideo"] = df["ideo"].replace([" ", "0", "9"], np.nan).transform(lambda x: int(x) - 4 if not pd.isnull(x) else np.nan)
df["weightz"] = pd.to_numeric(df["weightz"].replace([" "], np.nan))
df["thermo_liberals"] = pd.to_numeric(df["thermo_liberals"].replace([" ", "98", "99"], np.nan))
df["thermo_conservatives"] = pd.to_numeric(df["thermo_conservatives"].replace([" ", "98", "99"], np.nan))
df["thermo_democratparty"] = pd.to_numeric(df["thermo_democratparty"].replace([" ", "98", "99"], np.nan))
df["thermo_republicanparty"] = pd.to_numeric(df["thermo_republicanparty"].replace([" ", "98", "99"], np.nan))

df["abram_civil_aid_to_blacks"] = pd.to_numeric(df["abram_civil_aid_to_blacks"].replace([" ", "0", "9", "1", "2", "3", "4", "5", "6", "7"], [np.nan, np.nan, 0, -1, -1, -1, 0, 1, 1, 1]))
# Adding in copy of VCF0803 here
df["abram_ideo"] = pd.to_numeric(orig_ideo.replace([" ", "0", "9", "1", "2", "3", "4", "5", "6", "7"], [np.nan, np.nan, 0, -1, -1, -1, 0, 1, 1, 1]))
df["abram_foreign_defense_spending"] = pd.to_numeric(df["abram_foreign_defense_spending"].replace([" ", "0", "9", "1", "2", "3", "4", "5", "6", "7"], [np.nan, np.nan, 0, -1, -1, -1, 0, 1, 1, 1]))
df["abram_econ_guarantee_jobs"] = pd.to_numeric(df["abram_econ_guarantee_jobs"].replace([" ", "0", "9", "1", "2", "3", "4", "5", "6", "7"], [np.nan, np.nan, 0, -1, -1, -1, 0, 1, 1, 1]))
df["abram_econ_health_insurance"] = pd.to_numeric(df["abram_econ_health_insurance"].replace([" ", "0", "9", "1", "2", "3", "4", "5", "6", "7"], [np.nan, np.nan, 0, -1, -1, -1, 0, 1, 1, 1]))
df["abram_econ_gov_services"] = pd.to_numeric(df["abram_econ_gov_services"].replace([" ", "0", "9", "1", "2", "3", "4", "5", "6", "7"], [np.nan, np.nan, 0, 1, 1, 1, 0, -1, -1, -1]))
df["abram_moral_abortion_allowed_law"] = pd.to_numeric(df["abram_moral_abortion_allowed_law"].replace([" ", "0", "9", "1", "2", "3", "4"], [np.nan, np.nan, 0, 1, 1, 0, -1]))

In [50]:
# Affect - Ideological

data = df.dropna(subset=["party", "ideo", "thermo_liberals", "thermo_conservatives", "weightz"])
data = data[(data["ideo"] >= -3) & (data["ideo"] <= 3) & (data["ideo"] != 0)]
dem = (data["ideo"] < 0)
rep = (data["ideo"] > 0)

data.insert(0, "polar", None)
data.loc[dem, "polar"] = data.loc[dem, "thermo_liberals"] - data.loc[dem, "thermo_conservatives"]
data.loc[rep, "polar"] = data.loc[rep, "thermo_conservatives"] - data.loc[rep, "thermo_liberals"]

data_r = data[rep]
data_d = data[dem]

wm = lambda x: np.average(x["polar"], weights=data.loc[x["polar"].index, "weightz"])
avg_data_r = data_r.groupby("year").apply(wm)
avg_data_d = data_d.groupby("year").apply(wm)
avg_data = pd.concat([avg_data_r, avg_data_d]).groupby("year").apply(sum)

data = pd.concat([data_r, data_d])
number_of_responses_by_year = data.groupby("year").size()
errorbar = {}
for d in data.values:
    polar = d[0]
    year = int(d[1])
    weight = d[4]
    if year not in errorbar:
        errorbar[year] = 0
    errorbar[year] += np.square(weight / number_of_responses_by_year[year]) * avg_data[year]

for year in errorbar:
    errorbar[year] = np.sqrt(errorbar[year])

x = np.array([int(a) for a in dict(avg_data).keys()])
y1 = list(dict(avg_data).values())
running['Affect - Ideological'] = y1
years_dict['Affect - Ideological'] = x


In [51]:
# Affect - Party

datap = df.dropna(subset=["party", "ideo", "thermo_democratparty", "thermo_republicanparty", "weightz"])
datap = datap[(datap["party"] >= -3) & (datap["party"] <= 3) & (datap["party"] != 0)]
dem = (datap["party"] < 0)
rep = (datap["party"] > 0)

datap.insert(0, "polar", None)
dem_in_group = datap.loc[dem, "thermo_democratparty"]
dem_out_group = datap.loc[dem, "thermo_republicanparty"]
rep_in_group = datap.loc[rep, "thermo_republicanparty"]
rep_out_group = datap.loc[rep, "thermo_democratparty"]
datap.loc[dem, "polar"] = dem_in_group - dem_out_group
datap.loc[rep, "polar"] = rep_in_group - rep_out_group

datap_r = datap[rep]
datap_d = datap[dem]

wmp = lambda x: np.average(x["polar"], weights=datap.loc[x["polar"].index, "weightz"])
wmp_dem_party = lambda x: np.average(x["thermo_democratparty"], weights=datap.loc[x["thermo_democratparty"].index, "weightz"])
wmp_rep_party = lambda x: np.average(x["thermo_republicanparty"], weights=datap.loc[x["thermo_republicanparty"].index, "weightz"])
avg_datap_r = datap_r.groupby("year").apply(wmp)
avg_datap_d = datap_d.groupby("year").apply(wmp)

avg_datap_dem_in_group = datap_d.groupby("year").apply(wmp_dem_party).groupby("year").apply(sum)
avg_datap_dem_out_group = datap_d.groupby("year").apply(wmp_rep_party).groupby("year").apply(sum)
avg_datap_rep_in_group = datap_r.groupby("year").apply(wmp_rep_party).groupby("year").apply(sum)
avg_datap_rep_out_group = datap_r.groupby("year").apply(wmp_dem_party).groupby("year").apply(sum)

avg_datap = pd.concat([avg_datap_r, avg_datap_d]).groupby("year").apply(sum)

data = pd.concat([datap_r, datap_d])
number_of_responses_by_year = data.groupby("year").size()
errorbar = {}
for d in data.values:
    polar = d[0]
    year = int(d[1])
    weight = d[4]
    if year not in errorbar:
        errorbar[year] = 0
    errorbar[year] += np.square(weight / number_of_responses_by_year[year]) * avg_datap[year]

for year in errorbar:
    errorbar[year] = np.sqrt(errorbar[year])

x = np.array([int(a) for a in dict(avg_datap).keys()])
y2 = list(dict(avg_datap).values())
running['Affect - Party'] = y2
years_dict['Affect - Party'] = x


In [52]:
# Party Sorting

data = df.dropna(subset=["party", "ideo", "weightz"]).loc[:, ["year", "party", "ideo", "weightz"]]
overlap = abs(data["party"] - data["ideo"]) + 1
max_overlap = max(overlap)
min_overlap = min(overlap)
reverse_overlap = (max_overlap + min_overlap) - overlap

party_str = abs(data["party"]) + 1
ideo_str = abs(data["ideo"]) + 1
party_ideo_str = party_str * ideo_str

overlap_str = reverse_overlap * party_ideo_str
data["polar"] = (overlap_str - 7) / 105 # Boxell 2017 Note: Renormalizes between 0 and 1 (same as what Mason (2013) does)

wm = lambda x: np.average(x["polar"], weights=data.loc[x.index, "weightz"])
avg_data = data.groupby("year").apply(wm)

number_of_responses_by_year = data.groupby("year").size()
errorbar = {}
for d in data.values:
    year = d[0]
    weight = d[3]
    if year not in errorbar:
        errorbar[year] = 0
    errorbar[year] += np.square(weight / number_of_responses_by_year[year]) * avg_data[year]

for year in errorbar:
    errorbar[year] = np.sqrt(errorbar[year])

x = np.array([int(a) for a in dict(avg_data).keys()])
y3 = list(dict(avg_data).values())
running['Party Sorting'] = y3
years_dict['Party Sorting'] = x


In [53]:
# Issue Consistency

data = df.dropna(subset=["abram_ideo", "abram_civil_aid_to_blacks", "abram_foreign_defense_spending", "abram_econ_guarantee_jobs", "abram_econ_health_insurance", "abram_econ_gov_services", "abram_moral_abortion_allowed_law"]).loc[:, ["year", "weightz", "abram_ideo", "abram_civil_aid_to_blacks", "abram_foreign_defense_spending", "abram_econ_guarantee_jobs", "abram_econ_health_insurance", "abram_econ_gov_services", "abram_moral_abortion_allowed_law"]]

data["polar"] = abs(data[["abram_ideo", "abram_civil_aid_to_blacks", "abram_foreign_defense_spending", "abram_econ_guarantee_jobs", "abram_econ_health_insurance", "abram_econ_gov_services", "abram_moral_abortion_allowed_law"]].sum(axis=1))

wm = lambda x: np.average(x["polar"], weights=data.loc[x.index, "weightz"])
avg_data = data.groupby("year").apply(wm)

In [54]:

number_of_responses_by_year = data.groupby("year").size()
errorbar = {}
for d in data.values:
    year = d[0]
    weight = d[1]
    if year not in errorbar:
        errorbar[year] = 0
    errorbar[year] += np.square(weight / number_of_responses_by_year[year]) * avg_data[year]

for year in errorbar:
    errorbar[year] = np.sqrt(errorbar[year])

x = np.array([int(a) for a in dict(avg_data).keys()])
y4 = list(dict(avg_data).values())
running['Issue Consistency'] = y4
years_dict['Issue Consistency'] = x


In [55]:
# Issue Divergence

issues = ["abram_ideo", "abram_civil_aid_to_blacks", "abram_foreign_defense_spending", "abram_econ_guarantee_jobs", "abram_econ_health_insurance", "abram_econ_gov_services", "abram_moral_abortion_allowed_law"]
data = df.loc[df["year"] > 1980, ["year", "weightz", "party", *issues]]

data.insert(0, "abram_party", np.nan)
data.loc[(data["party"] > 0), "abram_party"] = 1
data.loc[(data["party"] < 0), "abram_party"] = 0
data = data.dropna()

wm = lambda x: st.kendalltau(x["abram_party"], x["abram_ideo"]).correlation
avg_data = data.groupby("year").apply(wm)

for issue in issues[1:]:
    wm = lambda x: st.kendalltau(x["abram_party"], x[issue]).correlation
    avg_data += data.groupby("year").apply(wm)

avg_data = avg_data / len(issues)

In [56]:
number_of_responses_by_year = data.groupby("year").size()
errorbar = {}
for d in data.values:
    year = d[1]
    weight = d[2]
    if year not in errorbar:
        errorbar[year] = 0
    errorbar[year] += np.square(weight / number_of_responses_by_year[year]) * avg_data[year]

for year in errorbar:
    errorbar[year] = np.sqrt(errorbar[year])

x = np.array([int(a) for a in dict(avg_data).keys()])
y5 = list(dict(avg_data).values())
running['Issue Divergence'] = y5
years_dict['Issue Divergence'] = x


In [59]:
for item in running.keys():
    print(str(item) + " : " + str(len(running[item])))
    print(years_dict[item])




Chamber Polarization DWNominate : 21
51    1981
52    1983
53    1985
54    1987
55    1989
56    1991
57    1993
58    1995
59    1997
60    1999
61    2001
62    2003
63    2005
64    2007
65    2009
66    2011
67    2013
68    2015
69    2017
70    2019
71    2021
Name: year, dtype: int64
Affect - Ideological : 20
[1972 1974 1976 1980 1982 1984 1986 1988 1990 1992 1994 1996 1998 2000
 2002 2004 2008 2012 2016 2020]
Affect - Party : 17
[1978 1980 1982 1984 1986 1988 1990 1992 1994 1996 1998 2000 2004 2008
 2012 2016 2020]
Party Sorting : 21
[1972 1974 1976 1978 1980 1982 1984 1986 1988 1990 1992 1994 1996 1998
 2000 2002 2004 2008 2012 2016 2020]
Issue Consistency : 10
[1984 1988 1992 1994 1996 2000 2004 2008 2016 2020]
Issue Divergence : 10
[1984 1988 1992 1994 1996 2000 2004 2008 2016 2020]
